# Streaming

Streaming is an important UX consideration for LLM apps, and agents are no exceptions. Streaming with agents is made more complicated by the fact that it's not just tokens that you will want to stream, but you may also want to stream back the intermediate steps an agent takes.

Let's take a look at how to do this.

## Set up the agent

Let's set up a simple agent for demonstration purposes

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
tools = [search]

In [5]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.chat_models import ChatOpenAI

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

## Stream intermediate steps

Let's look at how to stream intermediate steps. We can do this easily by just using the `.stream` method on the AgentExecutor

In [9]:
for chunk in agent_executor.stream({"input": "what is the weather in SF and then LA"}):
    print(chunk)
    print("------")

{'actions': [AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{\n  "query": "weather in San Francisco"\n}'}})])], 'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{\n  "query": "weather in San Francisco"\n}'}})]}
------
{'steps': [AgentStep(action=AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{\n  "query": "weather in San Francisco"\n}'}})]), observation=[{'

You can see that we get back a bunch of different information. There are two ways to work with this information:

1. By using the AgentAction/observation/AgentFinish object
2. By using the `messages` object

### Using AgentAction/observation/AgentFinish

You can access these raw objects as part of the streamed payload. This gives you more low level information, but can be harder to parse.

In [11]:
for chunk in agent_executor.stream({"input": "what is the weather in SF and then LA"}):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(
                f"Calling Tool ```{action.tool}``` with input ```{action.tool_input}```"
            )
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Got result: ```{step.observation}```")
    # Final result
    elif "output" in chunk:
        print(chunk["output"])
    else:
        raise ValueError
    print("------")

Calling Tool ```tavily_search_results_json``` with input ```{'query': 'weather in San Francisco'}```
------
Got result: ```[{'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US', 'content': 'recents Specialty Forecasts 10 Day Weather-San Francisco, CA Today Mon 18 | Day  Fri 22 Fri 22 | Day Foggy early, then partly cloudy later in the day. High around 60F. Winds W at 10 to 15 mph.  Considerable cloudiness with occasional rain showers. High 59F. Winds SSE at 5 to 10 mph. Chance of rain 50%.  Thu 28 | Night Cloudy with showers. Low 46F. Winds S at 5 to 10 mph. Chance of rain 40%. Fri 29 Fri 29 | DaySan Francisco, CA 10-Day Weather Forecast - The Weather Channel | Weather.com 10 Day Weather - San Francisco, CA As of 12:09 pm PST Today 60°/ 54° 23% Tue 19 | Day 60° 23% S 12 mph...'}]```
------
Calling Tool ```tavily_search_results_json``` with input ```{'query': 'weather in Los Angeles'}```
------
Got result: ```[{'url': 'https://ktla.com/weather/', 'content': 'Cali

### Using messages

Using messages can be nice when working with chat applications - because everything is a message!

In [12]:
for chunk in agent_executor.stream({"input": "what is the weather in SF and then LA"}):
    print(chunk["messages"])
    print("------")

[AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{\n  "query": "weather in San Francisco"\n}'}})]
------
[FunctionMessage(content='[{"url": "https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US", "content": "recents Specialty Forecasts 10 Day Weather-San Francisco, CA Today Mon 18 | Day  Fri 22 Fri 22 | Day Foggy early, then partly cloudy later in the day. High around 60F. Winds W at 10 to 15 mph.  Considerable cloudiness with occasional rain showers. High 59F. Winds SSE at 5 to 10 mph. Chance of rain 50%.  Thu 28 | Night Cloudy with showers. Low 46F. Winds S at 5 to 10 mph. Chance of rain 40%. Fri 29 Fri 29 | DaySan Francisco, CA 10-Day Weather Forecast - The Weather Channel | Weather.com 10 Day Weather - San Francisco, CA As of 12:09 pm PST Today 60°/ 54° 23% Tue 19 | Day 60° 23% S 12 mph..."}]', name='tavily_search_results_json')]
------
[AIMessage(content='', additional_kwargs={'function_call': {'name': '

## Stream tokens

In addition to streaming the final result, you can also stream tokens. This will require slightly more complicated parsing of the logs

You will also need to make sure you set the LLM to be streaming

In [14]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [15]:
async for chunk in agent_executor.astream_log(
    {"input": "what is the weather in sf", "chat_history": []},
    include_names=["ChatOpenAI"],
):
    print(chunk)

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '6b2e6e19-b9e4-4a6f-a527-5200bfb6b3a5',
            'logs': {},
            'streamed_output': []}})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI',
  'value': {'end_time': None,
            'final_output': None,
            'id': '32831c05-47b8-4381-8539-b3215d585b03',
            'metadata': {},
            'name': 'ChatOpenAI',
            'start_time': '2023-12-24T03:07:49.142',
            'streamed_output_str': [],
            'tags': ['seq:step:3'],
            'type': 'llm'}})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''})
RunLogPatch({'op': 'add', 'path': '/logs/ChatO

RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': ' around'})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': ' '})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': '60'})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': '°F'})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': '.'})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': ' Later'})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': ' in'})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': ' the'})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': ' day'})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': ','})
RunLogPat

This may require some logic to get in a workable format

In [24]:
path_status = {}
async for chunk in agent_executor.astream_log(
    {"input": "what is the weather in sf", "chat_history": []},
    include_names=["ChatOpenAI"],
):
    for op in chunk.ops:
        if op["op"] == "add":
            if op["path"] not in path_status:
                path_status[op["path"]] = op["value"]
            else:
                path_status[op["path"]] += op["value"]
    print(op["path"])
    print(path_status.get(op["path"]))
    print("----")


None
----
/logs/ChatOpenAI
{'id': '4c3ff5de-7c29-4868-932b-7ab546b4bb5e', 'name': 'ChatOpenAI', 'type': 'llm', 'tags': ['seq:step:3'], 'metadata': {}, 'start_time': '2023-12-24T03:15:26.938', 'streamed_output_str': [], 'final_output': None, 'end_time': None}
----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/streamed_output_str/-

----
/logs/ChatOpenAI/end_time
2023-12-24T03:15:27.595
----
/final_output
None
----
/final_output/messages/1
c